In [1]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup

In [88]:
def get_url(search_item, page):
    """Search the item using the url."""

    init_url = "https://www.amazon.in/s?k={}"
    search_item = search_item.replace(" ", "+")
    final_url = init_url.format(search_item)
    final_url = final_url + "&page={}"
    return final_url.format(page)

def extractRow(item):
    """Extract details of each row of Amazon Product."""

    atag = item.h2.a
    title = atag.text.strip()
    url = 'https://www.amazon.in' + atag.get('href')

    try:
        price = item.find('span', 'a-price')
        price = price.find('span', 'a-offscreen').text
    except AttributeError:
        return 
    
    try:
        ratings = item.i.text
        review_count = item.find('span', {'class': 'a-size-base'}).text
    except AttributeError:
        ratings = ''
        review_count = ''

    return (title, price, ratings, review_count, url)

records = list()

def main(search):
    """Executes the main program."""

    # Initialize the Chrome driver
    driver = webdriver.Chrome("chromedriver")

    record = list()

    for page in range(1, 21):
        driver.get(get_url(search, page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.findAll('div', {'data-component-type': 's-search-result'})

        for result in results:
            record = extractRow(result)

            if record:
                records.append(record)

    driver.close()

    # # Saving file to a CSV
    data = pd.DataFrame(data=records, columns=['Title', 'Price', 'Ratings', 'Review Count', 'URL'])
    data.to_csv('results.csv', index=False)

In [89]:
# Search and get the data associated with the product
search = input("Enter the Product:")
main(search=search)